# 运行试验

可以使用 Azure 机器学习 SDK 运行记录指标并生成输出的代码试验。这是 Azure 机器学习中大多数机器学习操作的核心。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**： 如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 运行试验

数据科学家需执行的最基本任务之一就是创建并运行可处理和分析数据的试验。在本练习中，你将学习如何使用 Azure ML 试验运行 Python *代码* 并记录从数据中提取的值。这种情况下，你将使用一个简单数据集，其中包含已进行糖尿病检测的患者的详细信息。你将运行试验以探索数据、提取统计信息、可视化效果和数据样本。你将使用的大多数代码是相当通用的 Python，就像你在任何数据探索过程中运行的那样。但需额外增加几行代码以使用 Azure ML *试验* 记录运行的详细信息。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# 在工作区中创建 Azure ML 试验
experiment = Experiment(workspace = ws, name = "diabetes-experiment")

# 开始记录试验中的数据
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 从本地文件加载数据
data = pd.read_csv('data/diabetes.csv')

# 计算行数并记录结果
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# 绘制并记录糖尿病患者和非糖尿病患者的计数
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name = 'label distribution', plot = fig)

# 记录各自妊娠计数
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# 记录数字列的摘要统计信息
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat = keys[index], value = values[index])
        
# 保存数据样本并将其上传到试验输出
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# 完成运行
run.complete()

## 查看试验结果

试验结束后，可以使用 **run** 对象获取运行及其输出的相关信息：

In [ ]:
import json

# 获取运行详细信息
details = run.get_details()
print(details)

# 获取记录的指标
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# 获取输出文件
files = run.get_file_names()
print(json.dumps(files, indent=2))

在 Jupyter Notebooks 中，可以使用 **“RunDetails”** 小组件，更好地可视化运行详细信息。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

请注意，**“RunDetails”** 小组件包含一个链接，用于查看 Azure 机器学习工作室中的运行。单击此链接，打开包含运行详细信息的新浏览器标签页（也可以直接打开 [Azure 机器学习工作室](https://ml.azure.com)，在 **“试验”** 页中查找运行）。在 Azure 机器学习工作室中查看运行时，请注意以下内容：

-**“属性”** 选项卡包含试验运行的常规属性。
-**“指标”** 选项卡支持选择已记录指标并以表格或图表形式查看。
-**“图像”** 选项卡支持选择和查看试验中记录的所有图像或绘图（本例中为 *“标签分布”* 绘图）
-**“子运行”** 选项卡列出所有子运行（本试验中没有）。
-**“输出”** 选项卡显示试验生成的输出文件。
-**“日志”** 选项卡显示由计算上下文针对试验生成的所有日志（本例中，实验内联运行，因此没有日志）。
-**“快照”** 选项卡包含运行试验代码的文件夹中的所有文件（本例中，所有文件与此笔记本位于同一文件夹）。
-**“Raw JSON”** 选项卡显示试验详细信息的 JSON 表示法。
-**“说明”** 选项卡可显示试验生成的模型说明（本例中没有）。

## 运行试验脚本

上一示例中，你在此笔记本中内联运行了一个试验。更为灵活的解决方案如下：为试验创建单独的脚本，将其与其所需的其他所有文件一同存储在一个文件夹中，然后使用 Azure ML 基于文件夹中的脚本运行试验。

首先，为试验文件创建文件夹，然后将数据复制到其中：

In [ ]:
import os, shutil

# 为试验文件创建文件夹
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# 将数据文件复制到试验文件夹
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

接下来，创建包含试验代码的 Python 脚本，然后将其保存到试验文件夹。

> **注意**： 运行以下单元格会直接 *创建* 脚本文件 - 不会运行！

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据集
data = pd.read_csv('diabetes.csv')

# 计算行数并记录结果
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# 计算并记录标签计数
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# 将数据样本保存到输出文件夹（自动上传）
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# 完成运行
run.complete()

此代码是之前使用的内联代码的简化版本。但请注意以下内容：
- 运行脚本时，它使用 `Run.get_context()` 方法检索试验运行上下文。
- 它从脚本所在文件夹加载糖尿病数据。
- 它创建名为 **outputs** 的文件夹，并将示例文件写入其中 - 该文件夹会自动上传到试验运行

现在基本可以进行试验。只需解决以下配置问题：

1.创建 *“运行配置”*，用于定义脚本的 Python 代码执行环境 - 本例中，它将自动创建安装有一些默认 Python 包的 Conda 环境。
2.创建 *“脚本配置”*，用于标识要在试验中运行的 Python 脚本文件以及运行该文件的环境。

> **注意**：目前不必太过担心环境配置 - 后续我们将进行更深入的探讨。

以下单元格可设置这些配置对象，然后提交试验。

In [ ]:
import os
import sys
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig
from azureml.widgets import RunDetails

# 创建新的 RunConfig 对象
experiment_run_config = RunConfiguration()

# 创建脚本配置
src = ScriptRunConfig(source_directory=experiment_folder, 
                      script='diabetes_experiment.py',
                      run_config=experiment_run_config) 

# 提交试验
experiment = Experiment(workspace = ws, name = 'diabetes-experiment')
run = experiment.submit(config=src)
RunDetails(run).show()
run.wait_for_completion()

和以前一样，可以使用小组件或指向 [Azure 机器学习工作室](https://ml.azure.com)中的试验的链接来查看试验生成的输出，还可以编写代码来检索其生成的指标和文件：

In [ ]:
# 获取记录的指标
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 查看试验运行历史记录

多次运行同一试验后，可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看历史记录并浏览所记录的每个运行。或者，可以从工作区按名称检索试验，然后使用 SDK 遍历其运行：

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['diabetes-experiment']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

> **更多信息**： 若要了解有关运行试验的更多信息，请参阅 Azure ML 文档中的[本主题](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs)。有关如何在运行期间记录指标的详细信息，请参阅[本主题](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments)。